!!! Before running this, masks should be exported in coco format, stored in 'ADHD-28.json'
- import all the original image to the folder 'data/original'
- clean the 'data/mask' folder

### Feature extraction using pyradiomics

In [23]:
from __future__ import print_function
import os  # needed navigate the system to get the input data
import SimpleITK as sitk
from radiomics import featureextractor
import numpy as np
import pandas as pd
from pycocotools import mask as maskUtils
from pycocotools.coco import COCO
import pandas as pd

In [24]:
import json
from matplotlib import pyplot as plt

In [25]:
# Change the following path to the location of the input data
with open('thalamus-coco.json') as f:
    data = json.load(f)

In [26]:
data.keys()

dict_keys(['images', 'categories', 'annotations'])

In [27]:
coco_img = data['images']
coco_cat = data['categories']
coco_segmentation = data['annotations']

In [28]:
print(coco_segmentation[0].keys())
print(coco_img[0].keys())

dict_keys(['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'isbbox', 'color', 'metadata'])
dict_keys(['id', 'dataset_id', 'category_ids', 'path', 'width', 'height', 'file_name', 'annotated', 'annotating', 'num_annotations', 'metadata', 'deleted', 'milliseconds', 'events', 'regenerate_thumbnail'])


In [29]:
coco_img[0]['file_name']

'P0006_morph1_0000000D.png'

In [30]:
maskPath = os.path.join(os.getcwd(), 'data','mask')

In [31]:
thalamus_maskPath = os.path.join(maskPath, 'thalamus')
cerebellum_maskPath = os.path.join(maskPath, 'cerebellum')
os.makedirs(thalamus_maskPath, exist_ok=True)
os.makedirs(cerebellum_maskPath, exist_ok=True)

In [32]:
for i in range(len(coco_segmentation)):
    width = coco_img[i]['width']
    height = coco_img[i]['height']
    img_name = coco_img[i]['file_name']
    segmentation = coco_segmentation[i]['segmentation']
    rle = maskUtils.frPyObjects(segmentation, height, width)
    mymask = maskUtils.decode(rle)
    mymask = np.max(mymask, axis=-1)*255
    #plt.imshow(mask)
    sitk_image = sitk.GetImageFromArray(mymask.astype(np.uint8))
    sitk.WriteImage(sitk_image, os.path.join(thalamus_maskPath, img_name))
#print (data_mask)
# data_mask is a dictionary with the image name as the key and the mask array as the value

In [33]:
coco = COCO('/Users/maicuigua/Documents/a Study/usyd master/Capstone/Project_dir/Thalamus_Jingtong/thalamus-coco.json')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [34]:
wd= os.getcwd()

In [35]:
imagePath = os.path.join(os.getcwd(), 'data','original')
paramPath = '/Users/maicuigua/Documents/a Study/usyd master/Capstone/Project_dir/Thalamus_Jingtong/myParams.yaml'

In [36]:
# haha = sitk.GetImageFromArray(data_mask['P0001_morph1_00000023.png'])
# haha.GetPixelIDTypeAsString()

## Loop through images and masks, extract featues

In [37]:
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)

all_features = {}
# Loop over images,exract features
for i in range(len(coco_img)):
    imagename= coco_img[i]['file_name']
    #check filename here
    image = sitk.ReadImage(os.path.join(imagePath, imagename))
    if image.GetNumberOfComponentsPerPixel() > 1:
         image = sitk.VectorMagnitude(image)
    
    # Read the mask and convert to binary if it is not already
    masks = sitk.ReadImage(os.path.join(thalamus_maskPath, imagename))
    
    all_features[imagename] = extractor.execute(image,masks)



GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

In [38]:
coco_img[0]['file_name']

'P0006_morph1_0000000D.png'

In [39]:
all_features['P0006_morph1_0000000D.png']

OrderedDict([('diagnostics_Versions_PyRadiomics', '0+unknown'),
             ('diagnostics_Versions_Numpy', '1.26.4'),
             ('diagnostics_Versions_SimpleITK', '2.3.1-g42ce2'),
             ('diagnostics_Versions_PyWavelet', '1.6.0'),
             ('diagnostics_Versions_Python', '3.11.8'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': True,
               'normalizeScale': 100,
               'removeOutliers': None,
               'resampledPixelSpacing': None,
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 255,
               'additionalInfo': True,
               'binWidth': 25,
               'weightingNorm': None}),
             ('dia

In [40]:
#save needed feature names into a list    
feature_names = list(filter(lambda x: x.startswith('original_'), all_features['P0006_morph1_0000000D.png']))
feature_names

['original_firstorder_10Percentile',
 'original_firstorder_90Percentile',
 'original_firstorder_Energy',
 'original_firstorder_Entropy',
 'original_firstorder_InterquartileRange',
 'original_firstorder_Kurtosis',
 'original_firstorder_Maximum',
 'original_firstorder_MeanAbsoluteDeviation',
 'original_firstorder_Mean',
 'original_firstorder_Median',
 'original_firstorder_Minimum',
 'original_firstorder_Range',
 'original_firstorder_RobustMeanAbsoluteDeviation',
 'original_firstorder_RootMeanSquared',
 'original_firstorder_Skewness',
 'original_firstorder_TotalEnergy',
 'original_firstorder_Uniformity',
 'original_firstorder_Variance',
 'original_glcm_Autocorrelation',
 'original_glcm_ClusterProminence',
 'original_glcm_ClusterShade',
 'original_glcm_ClusterTendency',
 'original_glcm_Contrast',
 'original_glcm_Correlation',
 'original_glcm_DifferenceAverage',
 'original_glcm_DifferenceEntropy',
 'original_glcm_DifferenceVariance',
 'original_glcm_Id',
 'original_glcm_Idm',
 'original_glc

In [41]:
#Create a numpy array to store the features, each row is an image, each column is a feature
samples = np.zeros((len(coco_img),len(feature_names)))

for idx,img_name in enumerate(all_features.keys()):
    a = np.array([])
    for feature_name in feature_names:
        a = np.append(a, all_features[img_name][feature_name])
    samples[idx,:] = a

#samples = np.nan_to_num(samples)
samples

array([[-6.95311472e+01, -4.25997927e+01,  2.45862272e+06, ...,
         2.27880565e-02,  4.85123538e-02,  1.73717815e-02],
       [-7.26497734e+01, -3.87360388e+01,  3.18601586e+06, ...,
         2.56885076e-02,  6.45161725e-02,  1.75607299e-02],
       [-8.53800560e+01, -4.62012727e+01,  2.78042893e+06, ...,
         3.09316818e-02,  1.43588139e-01,  1.45385973e-02],
       ...,
       [-8.94821460e+01, -2.77174023e+01,  1.76227716e+06, ...,
         4.81094368e-02,  5.84774604e-01,  7.47136551e-03],
       [-1.00304943e+02, -6.28289841e+01,  4.58587012e+06, ...,
         3.07724715e-02,  1.96951128e-01,  9.47793239e-03],
       [-6.86775850e+01, -4.08759180e+01,  1.59034811e+06, ...,
         3.44098314e-02,  1.14379375e-01,  2.12262015e-02]])

In [42]:
# Convert the array to a DataFrame
df = pd.DataFrame(samples,columns=feature_names)

# add a column to store the image name
df['img_name']=list(all_features.keys())



In [43]:
df.head()

,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Mean,original_firstorder_Median,...,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,img_name
0,-69.531147,-42.599793,2.458623e+06,0.806056,17.505380,3.900235,-14.321871,10.059310,-60.849052,-69.531147,...,0.194749,1.688576,62.007825,92.571205,54.591288,0.840028,0.022788,0.048512,0.017372,P0006_morph1_0000000D.png
1,-72.649773,-38.736039,3.186016e+06,0.809481,17.635142,4.136352,-14.318150,11.058262,-61.690148,-68.580125,...,0.216914,1.695107,63.240903,94.407779,56.415307,0.852712,0.025689,0.064516,0.017561,P0015_morph1_0000000A.png
2,-85.380056,-46.201273,2.780429e+06,1.508378,19.589392,5.318450,-8.328449,13.061075,-68.767969,-76.238340,...,0.623571,3.357401,51.624549,146.317690,35.145094,0.611525,0.030932,0.143588,0.014539,P0020_morph1_00000001.png
3,-83.782248,-51.400871,1.661532e+06,1.430128,16.190688,5.102134,-10.301431,11.003333,-66.760098,-70.082435,...,0.504741,3.554930,47.039437,144.997183,25.433705,0.520403,0.039443,0.212202,0.016268,P0020_morph1_00000002.png
4,-72.141942,-2.196814,1.278204e+06,2.248570,39.423618,6.080969,94.454636,23.936800,-40.838652,-47.979079,...,1.753511,10.035789,41.113684,321.328421,9.935742,0.222930,0.045226,0.689137,0.008404,P0020_morph1_00000003.png


In [44]:
# Save the DataFrame to an Excel file
df.to_excel("thalamus_echogenicity_feature.xlsx", index=False)

In [45]:
# #check image type, dimension, shape
# print("Image:")
# print("Pixel Type:", image_1.GetPixelIDTypeAsString())
# print("Number of Components per Pixel:", image_1.GetNumberOfComponentsPerPixel())
# print("Image Dimensions:", image_1.GetDimension())
# print("Image Size:", image_1.GetSize())

# # Inspect mask attributes
# print("\nMask:")
# print("Pixel Type:", mask_1.GetPixelIDTypeAsString())
# print("Number of Components per Pixel:", mask_1.GetNumberOfComponentsPerPixel())
# print("Mask Dimensions:", mask_1.GetDimension())
# print("Mask Size:", mask_1.GetSize())